# Humans or Horses?
A tensorflow based binary classification model

Getting Dependencies

In [1]:
import tensorflow as tf
import os
import zipfile
import urllib

Downloading Dataset

In [7]:
urllib.request.urlretrieve("https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip",
                           filename="dataset/happy-or-sad.zip")
zip_ref = zipfile.ZipFile("dataset/happy-or-sad.zip", 'r')
zip_ref.extractall("dataset/h-or-s")
zip_ref.close()

In [11]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

    callbacks = myCallback()
    

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
        

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(
        'dataset/h-or-s/',
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

    history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1, callbacks=[callbacks])

    return history.history['acc'][-1]

Evaluating Our Model By Running Above Function

In [12]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 3s 388ms/step - loss: 1.8570 - acc: 0.4750
Epoch 2/15
8/8 [==============================] - 2s 223ms/step - loss: 0.4977 - acc: 0.8375
Epoch 3/15
8/8 [==============================] - 2s 260ms/step - loss: 0.2357 - acc: 0.8875
Epoch 4/15
8/8 [==============================] - 2s 242ms/step - loss: 0.0919 - acc: 0.9750
Epoch 5/15
8/8 [==============================] - 2s 235ms/step - loss: 0.1993 - acc: 0.9375
Epoch 6/15
8/8 [==============================] - 2s 226ms/step - loss: 0.0524 - acc: 0.9750
Epoch 7/15
8/8 [==============================] - 2s 222ms/step - loss: 0.0426 - acc: 0.9750
Epoch 8/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0076 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 224ms/step - loss: 0.0127 - acc: 1.0000


1.0